# Exercise 1

## Yanan Mo, Zimai Lian, Yu Chen Wu

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.linear_model import LinearRegression as LR
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima.model import ARIMA
from scipy import stats
from scipy.optimize import minimize 

In [2]:
bm_us = pd.read_excel('/Users/yananmo/Desktop/MGT 930/Excersie 1/BEME calculation.xlsx')

In [3]:
#drop row with Na values
bm_us = bm_us.dropna()

## Problem 1. US-ranking

### Setting variables and finding BEME

In [19]:
seq_us = bm_us['Stockholders Equity - Parent']
tax_us = bm_us['Deferred Taxes and Investment Tax Credit']
pfstock_us = bm_us['Preferred/Preference Stock (Capital) - Total']
csho_us = bm_us['Common Shares Outstanding']
price_closing_us = bm_us['Price Close - Annual - Calendar']

In [21]:
# book value=stock equity+tax-preferred stock
be_us = seq_us + tax_us - pfstock_us
#market value=ourstanding*price 
me_us = csho_us * price_closing_us
#BEME
beme_us = be_us/me_us


eliminate outlier from the return, if the return in a year is larger than 50, which means 5000%, we conclude that there might be data problem instead of real return change. Replacee the return with 1 in that period to elminate outlier effect.

In [304]:
ret=[]
for i in range(1,len(bm_us)):
    if bm_us.iloc[(i-1),7]==bm_us.iloc[(i),7]:
        y1=bm_us.iloc[(i-1),15]
        y2=bm_us.iloc[(i),15]
        ret1=(y2-y1)/y1
        if ret1>50:
            ret.append(1)
        else:
            ret.append(ret1)
    else:
        ret.append(0)

In [305]:
ret_us=[0]+list(ret)
np.array(ret_us)

array([ 0.        , -0.18367347, -0.425     , ...,  0.        ,
        0.        ,  0.        ])

In [306]:
bm_us['BEME'] = beme_us
bm_us['Return'] = ret_us

In [307]:
#new table based on only BEME 
bm_us1 = bm_us[['Data Year - Fiscal', 'Ticker Symbol', 'BEME',"Return"]]

In [308]:
bm_us1
bm_us1.to_excel(r"/Users/yananmo/Desktop/original.xlsx",index=True)

### Cleaning Data based on year

In [309]:
#have a list of individual stock name 
ticker_us = bm_us['Ticker Symbol']
ticker_us = list(dict.fromkeys(ticker_us))

In [310]:
a=[]
b=[]
c=[]
d=[]
for i in range(1950,2022):
    for f in range(0,len(bm_us1)):
        if float(bm_us1.iloc[f,0])==i:
            a.append(bm_us1.iloc[f,2])
            b.append(bm_us1.iloc[f,1])
            c.append(bm_us1.iloc[f,0])
            d.append(bm_us1.iloc[f,3])

In [311]:
Ctotal=pd.DataFrame({"ticker":b,"BEME":a,"year":c,"return":d})
Ctotal.head()

,ticker,BEME,year,return
0,Q2,1.053379,1950,0.000000
1,Q2,1.023725,1951,0.020096
2,Q2,1.007828,1952,0.014578
3,Q2,0.989562,1953,0.019417
4,Q2,0.875743,1954,0.128381


### Ranking based on BEME

Here, we drop all the data before 1961

In [312]:
c0 = Ctotal.groupby('year')
c1961 = c0.get_group(1961).sort_values(by=['BEME'],ascending=False).reset_index(drop=True) 

In [313]:
z1=c1961.iloc[0:int(len(c1961)/10)*3,[0,1,3]].reset_index(drop=True) 
z2=c1961.iloc[int(len(c1961)/10)*3:int(len(c1961)/10)*3+int(len(c1961)/10)*4,[0,1,3]].reset_index(drop=True) 
z3=c1961.iloc[int(len(c1961)/10)*3+int(len(c1961)/10)*4:,[0,1,3]].reset_index(drop=True) 

In [314]:
a=1961
ind=np.full(shape=len(z2),fill_value=a,dtype=np.int)

In [315]:
z1961=pd.concat([z1,z2,z3],axis=1)
z1961.columns=["High-name","High","high return","Middle-name","Middle","Middle Return","Low-name","Low","Low return"]
z1961.index=ind
z1961
z_exp=z1961
z_exp

,High-name,High,high return,Middle-name,Middle,Middle Return,Low-name,Low,Low return
1961,BRS.A1,9.947846,0.000000,MDU,0.517354,0.0,AEP,0.383143,0.0
1961,GNN,1.257895,0.191708,EIX1,0.514951,0.0,VVC,0.381281,0.0
1961,PVY,1.050569,0.000000,EIX,0.514951,0.0,VVC1,0.381281,0.0
1961,ES,0.773831,0.000000,SRE4,0.511228,0.0,0191A,0.377654,0.0
1961,2985B,0.768352,0.000000,ENA.1,0.511228,0.0,AES5,0.377654,0.0
...,...,...,...,...,...,...,...,...,...
1961,NaN,NaN,NaN,OKE,0.389490,0.0,NaN,NaN,NaN
1961,NaN,NaN,NaN,CEG.1,0.388207,0.0,NaN,NaN,NaN
1961,NaN,NaN,NaN,IPW.1,0.387750,0.0,NaN,NaN,NaN
1961,NaN,NaN,NaN,ILA,0.384773,0.0,NaN,NaN,NaN


### Ranking for all year and creating a excel file 

In [316]:
for i in range(1962,2022):
    c0 = Ctotal.groupby('year')
    cn = c0.get_group(i).sort_values(by=['BEME'],ascending=False).reset_index(drop=True) 
    z1=cn.iloc[0:int(len(cn)/10)*3,[0,1,3]].reset_index(drop=True) 
    z2=cn.iloc[int(len(cn)/10)*3:int(len(cn)/10)*3+int(len(cn)/10)*4,[0,1,3]].reset_index(drop=True) 
    z3=cn.iloc[int(len(cn)/10)*3+int(len(cn)/10)*4:,[0,1,3]].reset_index(drop=True) 
    f=i
    ind=np.full(shape=len(z2),fill_value=f,dtype=np.int)
    zn=pd.concat([z1,z2,z3],axis=1)
    zn.columns=["High-name","High","high return","Middle-name","Middle","Middle Return","Low-name","Low","Low return"]
    zn.index=ind
    z1961=pd.concat([z1961,zn])

In [149]:
z1961.to_excel(r"/Users/yananmo/Desktop/exercise1.xlsx",index=True)

### ranking based on ME only 

In [35]:
bm_us['ME'] = me_us
bm_us2 = bm_us[['Data Year - Fiscal', 'Ticker Symbol', 'ME']]
bm_us2

,Data Year - Fiscal,Ticker Symbol,ME
3,1964,AE.2,2.401000
9,1970,AE.2,24.460000
10,1971,AE.2,17.221250
11,1972,AE.2,14.872750
12,1973,AE.2,4.970000
...,...,...,...
484741,2021,STKH,67.161180
484742,2021,NXTP,92.691144
484744,2021,INDP,47.076300
484747,2021,QNRX,15.264340


In [36]:
c1 = bm_us2.groupby('Data Year - Fiscal')
cs = c1.get_group(1961).sort_values(by=['ME'],ascending=False).reset_index(drop=True) 

In [37]:
x1=cs.iloc[0:int(len(cs)/2),1:3].reset_index(drop=True) 
x2=cs.iloc[int(len(cs)/2):,1:3].reset_index(drop=True) 
a=1961
ind1=np.full(shape=len(x2),fill_value=a,dtype=np.int)

In [38]:
zs=pd.concat([x1,x2],axis=1)
zs.columns=["High-name","High","Low-name","Low"]
zs.index=ind1
zs

,High-name,High,Low-name,Low
1961,PCG,1927.311750,RGS.,155.543625
1961,UCM1,1808.301250,5495B,153.643500
1961,UCM.2,1808.301250,EVRG,150.100000
1961,AEP,1522.258500,OKE,150.034500
1961,ED,1321.309125,NFG,148.395750
...,...,...,...,...
1961,POR,183.833750,SWX,12.292000
1961,UTP,183.487500,SWEL,8.836500
1961,SRE4,171.000000,UTL1,8.591000
1961,ENA.1,171.000000,EDSE,4.369750


### Creating 6 portfolios

In [271]:
BH=[]
BHn=[]
BHr=[]
SH=[]
SHn=[]
SHr=[]
for i in range(0,len(z_exp)):
    if z_exp.iloc[i,0] in zs['High-name'].values:
        BH.append(z_exp.iloc[i,1])
        BHn.append(z_exp.iloc[i,0])
        BHr.append(z_exp.iloc[i,2])
        
    else:
        SH.append(z_exp.iloc[i,1])
        SHn.append(z_exp.iloc[i,0])
        SHr.append(z_exp.iloc[i,2])

In [272]:
BM=[]
BMn=[]
SM=[]
SMn=[]
BMr=[]
SMr=[]
for i in range(0,len(z_exp)):
    if z_exp.iloc[i,3] in zs['High-name'].values:
        BM.append(z_exp.iloc[i,4])
        BMn.append(z_exp.iloc[i,3])
        BMr.append(z_exp.iloc[i,5])
    else:
        SM.append(z_exp.iloc[i,4])
        SMn.append(z_exp.iloc[i,3])
        SMr.append(z_exp.iloc[i,5])

In [273]:
BL=[]
BLn=[]
SL=[]
SLn=[]
BLr=[]
SLr=[]
for i in range(0,len(z_exp)):
    if z_exp.iloc[i,6] in zs['High-name'].values:
        BL.append(z_exp.iloc[i,7])
        BLn.append(z_exp.iloc[i,6])
        BLr.append(z_exp.iloc[i,8])
    else:
        SL.append(z_exp.iloc[i,7])
        SLn.append(z_exp.iloc[i,6])
        SLr.append(z_exp.iloc[i,8])

In [274]:
CBH=pd.DataFrame({"B/H-name":BHn,"B/H":BH,"B/H-return":BHr}).reset_index(drop=True).dropna()
CSH=pd.DataFrame({"S/H-name":SHn,"S/H":SH,"S/H-return":SHr}).reset_index(drop=True).dropna()
CBM=pd.DataFrame({"B/M-name":BMn,"B/M":BM,"B/M-return":BMr}).reset_index(drop=True).dropna()
CSM=pd.DataFrame({"S/M-name":SMn,"S/M":SM,"S/M-return":SMr}).reset_index(drop=True).dropna()
CBL=pd.DataFrame({"B/L-name":BLn,"B/L":BL,"B/L-return":BLr}).reset_index(drop=True).dropna()
CSL=pd.DataFrame({"S/L-name":SLn,"S/L":SL,"S/L-return":SLr}).reset_index(drop=True).dropna()
Cb=pd.concat([CBH,CSH,CBM,CSM,CBL,CSL],axis=1)
year=1961   
ind2=np.full(shape=len(Cb.max(axis=1)),fill_value=year,dtype=np.int)
Cb.index=ind2
Cb

,B/H-name,B/H,B/H-return,S/H-name,S/H,S/H-return,B/M-name,B/M,B/M-return,S/M-name,S/M,S/M-return,B/L-name,B/L,B/L-return,S/L-name,S/L,S/L-return
1961,ES,0.773831,0.0,BRS.A1,9.947846,0.000000,MDU,0.517354,0.0,EIX1,0.514951,0.0,AEP,0.383143,0.0,VVC,0.381281,0.0
1961,NFG,0.732642,0.0,GNN,1.257895,0.191708,EIX,0.514951,0.0,SRE4,0.511228,0.0,XEL,0.372331,0.0,VVC1,0.381281,0.0
1961,OTTR,0.637166,0.0,PVY,1.050569,0.000000,CMS,0.505415,0.0,ENA.1,0.511228,0.0,NI,0.369579,0.0,0191A,0.377654,0.0
1961,ED,0.630464,0.0,2985B,0.768352,0.000000,ETR,0.489230,0.0,CPN.2,0.508326,0.0,SJI,0.368734,0.0,AES5,0.377654,0.0
1961,MGEE,0.617174,0.0,UTL1,0.764055,0.000000,PCG,0.469746,0.0,CMS1,0.505415,0.0,KMI,0.343177,0.0,NI1,0.369579,0.0
1961,SRE,0.613187,0.0,D2,0.743124,0.000000,EVRG,0.467368,0.0,WPS1,0.505054,0.0,HE,0.338176,0.0,ETR7,0.367184,0.0
1961,AVA,0.598157,0.0,CTP.2,0.717981,0.000000,IDA,0.459273,0.0,WEC3,0.505054,0.0,PNM,0.329352,0.0,AES4,0.364554,0.0
1961,BKH,0.574462,0.0,NST,0.688640,0.000000,EQT,0.441696,0.0,MWR2,0.504714,0.0,PNW,0.322763,0.0,DUK1,0.362585,0.0
1961,UGI,0.571220,0.0,UPEN,0.649362,0.000000,EXC,0.440893,0.0,IOR.1,0.504714,0.0,NJR,0.314424,0.0,DQE,0.356110,0.0
1961,ALE,0.571207,0.0,NU4,0.637014,0.000000,FE,0.439764,0.0,AEE3,0.500710,0.0,OGE,0.311428,0.0,DQE1,0.356110,0.0


### for all years

In [275]:
b1 = bm_us2.groupby('Data Year - Fiscal')
b2 = Ctotal.groupby('year')
for i in range(1962,2022):
    year=i
    #find ME in i year, ranking
    bs = b1.get_group(i).sort_values(by=['ME'],ascending=False).reset_index(drop=True) 
    #divid into two 
    x1=bs.iloc[0:int(len(bs)/2),1:3].reset_index(drop=True) 
    x2=bs.iloc[int(len(bs)/2):,1:3].reset_index(drop=True) 
    zs=pd.concat([x1,x2],axis=1)
    zs.columns=["High-name","High","Low-name","Low"]
    # find BEME in i year, ranking 
    cn = b2.get_group(i).sort_values(by=['BEME'],ascending=False).reset_index(drop=True) 
    z1=cn.iloc[0:int(len(cn)/10)*3,[0,1,3]].reset_index(drop=True) 
    z2=cn.iloc[int(len(cn)/10)*3:int(len(cn)/10)*3+int(len(cn)/10)*4,[0,1,3]].reset_index(drop=True) 
    z3=cn.iloc[int(len(cn)/10)*3+int(len(cn)/10)*4:,[0,1,3]].reset_index(drop=True) 
    zn=pd.concat([z1,z2,z3],axis=1)
    zn.columns=["High-name","High","high return","Middle-name","Middle","Middle Return","Low-name","Low","Low return"]
    # create BH and SH in year i
    BH2=[]
    BHn2=[]
    SH2=[]
    SHn2=[]
    BHr2=[]
    SHr2=[]
    for f in range(0,len(zn)):
        if zn.iloc[f,0] in zs['High-name'].values:
            BH2.append(zn.iloc[f,1])
            BHn2.append(zn.iloc[f,0])
            BHr2.append(zn.iloc[f,2])
        else:
            SH2.append(zn.iloc[f,1])
            SHn2.append(zn.iloc[f,0])
            SHr2.append(zn.iloc[f,2])
    # create BM and SM in year i
    BM2=[]
    BMn2=[]
    SM2=[]
    SMn2=[]
    BMr2=[]
    SMr2=[]
    g=0
    for g in range(0,len(zn)):
        if zn.iloc[g,3] in zs['High-name'].values:
            BM2.append(zn.iloc[g,4])
            BMn2.append(zn.iloc[g,3])
            BMr2.append(zn.iloc[g,5])
        else:
            SM2.append(zn.iloc[g,4])
            SMn2.append(zn.iloc[g,3]) 
            SMr2.append(zn.iloc[g,5])
    # create BL and SL in year i
    BL2=[]
    BLn2=[]
    SL2=[]
    SLn2=[]
    BLr2=[]
    SLr2=[]
    for h in range(0,len(zn)):
        if zn.iloc[h,6] in zs['High-name'].values:
            BL2.append(zn.iloc[h,7])
            BLn2.append(zn.iloc[h,6])
            BLr2.append(zn.iloc[h,8])
        else:
            SL2.append(zn.iloc[h,7])
            SLn2.append(zn.iloc[h,6])
            SLr2.append(zn.iloc[h,8])
    #create table 
    CBH2=pd.DataFrame({"B/H-name":BHn2,"B/H":BH2,"B/H-return":BHr2}).reset_index(drop=True).dropna() 
    CSH2=pd.DataFrame({"S/H-name":SHn2,"S/H":SH2,"S/H-return":SHr2}).reset_index(drop=True).dropna()
    CBM2=pd.DataFrame({"B/M-name":BMn2,"B/M":BM2,"B/M-return":BMr2}).reset_index(drop=True).dropna() 
    CSM2=pd.DataFrame({"S/M-name":SMn2,"S/M":SM2,"S/M-return":SMr2}).reset_index(drop=True).dropna()
    CBL2=pd.DataFrame({"B/L-name":BLn2,"B/L":BL2,"B/L-return":BLr2}).reset_index(drop=True).dropna()
    CSL2=pd.DataFrame({"S/L-name":SLn2,"S/L":SL2,"S/L-return":SLr2}).reset_index(drop=True).dropna()
    #combined into one 
    Cb2=pd.concat([CBH2,CSH2,CBM2,CSM2,CBL2,CSL2],axis=1)  
    ind2=np.full(shape=len(Cb2.max(axis=1)),fill_value=year,dtype=np.int)
    Cb2.index=ind2
    Cb=pd.concat([Cb,Cb2]) 
    BH2.clear()
    BHn2.clear()
    SH2.clear()
    SHn2.clear()
    BM2.clear()
    BMn2.clear()
    SM2.clear()
    SMn2.clear()
    BL2.clear()
    BLn2.clear()
    SL2.clear()
    SLn2.clear()
    BLr2.clear()
    SLr2.clear()
    BMr2.clear()
    SMr2.clear()
    BHr2.clear()
    SHr2.clear()

In [193]:
Cb.to_excel(r"/Users/yananmo/Desktop/6 portfolio5.xlsx",index=True)

## Problem 2

we will start at year 1964, where most of the company has passed their first year's record. 

In [276]:
Cb2=Cb.loc[1964:]
Cb2.reset_index(inplace=True)

In [278]:
BHm=[]
SHm=[]
BMm=[]
SMm=[]
BLm=[]
SLm=[]
cb22=Cb2.groupby("index")
for i in range(1964,2022):
    c1=cb22.get_group(i)
    c1m=c1.iloc[:,3].mean()
    c2m=c1.iloc[:,6].mean()
    c3m=c1.iloc[:,9].mean()
    c4m=c1.iloc[:,12].mean()
    c5m=c1.iloc[:,15].mean()
    c6m=c1.iloc[:,18].mean()
    BHm.append(c1m)
    SHm.append(c2m)
    BMm.append(c3m)
    SMm.append(c4m)
    BLm.append(c5m)
    SLm.append(c6m)

In [279]:
C5=pd.DataFrame({"B/H mean":BHm,"S/H mean":SHm,"B/M mean":BMm,"S/M mean":SMm,"B/L mean":BLm,"S/L mean":SLm})
ye=[]
for i in range(0,len(C5)):
    f=1964+i
    ye.append(f)
C5.index=ye

In [280]:
BHsd=[]
SHsd=[]
BMsd=[]
SMsd=[]
BLsd=[]
SLsd=[]
cb22=Cb2.groupby("index")
for i in range(1964,2022):
    c1=cb22.get_group(i)
    c1sd=c1.iloc[:,3].std()
    c2sd=c1.iloc[:,6].std()
    c3sd=c1.iloc[:,9].std()
    c4sd=c1.iloc[:,12].std()
    c5sd=c1.iloc[:,15].std()
    c6sd=c1.iloc[:,18].std()
    BHsd.append(c1sd)
    SHsd.append(c2sd)
    BMsd.append(c3sd)
    SMsd.append(c4sd)
    BLsd.append(c5sd)
    SLsd.append(c6sd)
C6=pd.DataFrame({"B/H sd":BHsd,"S/H sd":SHsd,"B/M sd":BMsd,"S/M sd":SMsd,"B/L sd":BLsd,"S/L sd":SLsd})
C6.index=ye

In [281]:
BHma=[]
SHma=[]
BMma=[]
SMma=[]
BLma=[]
SLma=[]
cb22=Cb2.groupby("index")
for i in range(1964,2022):
    c1=cb22.get_group(i)
    c1sd=c1.iloc[:,3].max()
    c2sd=c1.iloc[:,6].max()
    c3sd=c1.iloc[:,9].max()
    c4sd=c1.iloc[:,12].max()
    c5sd=c1.iloc[:,15].max()
    c6sd=c1.iloc[:,18].max()
    BHma.append(c1sd)
    SHma.append(c2sd)
    BMma.append(c3sd)
    SMma.append(c4sd)
    BLma.append(c5sd)
    SLma.append(c6sd)
C8=pd.DataFrame({"B/H max":BHma,"S/H max":SHma,"B/M max":BMma,"S/M max":SMma,"B/L max":BLma,"S/L max":SLma})
C8.index=ye

In [282]:
BHmi=[]
SHmi=[]
BMmi=[]
SMmi=[]
BLmi=[]
SLmi=[]
cb22=Cb2.groupby("index")
for i in range(1964,2022):
    c1=cb22.get_group(i)
    c1sd=c1.iloc[:,3].min()
    c2sd=c1.iloc[:,6].min()
    c3sd=c1.iloc[:,9].min()
    c4sd=c1.iloc[:,12].min()
    c5sd=c1.iloc[:,15].min()
    c6sd=c1.iloc[:,18].min()
    BHmi.append(c1sd)
    SHmi.append(c2sd)
    BMmi.append(c3sd)
    SMmi.append(c4sd)
    BLmi.append(c5sd)
    SLmi.append(c6sd)
C9=pd.DataFrame({"B/H min":BHmi,"S/H min":SHmi,"B/M min":BMmi,"S/M min":SMmi,"B/L min":BLmi,"S/L min":SLmi})
C9.index=ye

In [288]:
C10=pd.concat([C5,C6,C8,C9],axis=1)
C10.head()

,B/H mean,S/H mean,B/M mean,S/M mean,B/L mean,S/L mean,B/H sd,S/H sd,B/M sd,S/M sd,...,B/M max,S/M max,B/L max,S/L max,B/H min,S/H min,B/M min,S/M min,B/L min,S/L min
1964,0.022573,0.088632,0.064231,0.159575,0.082552,0.208098,0.253723,0.303170,0.241445,0.406131,...,1.025316,2.833333,1.152630,1.428571,-0.779412,-0.674569,-0.772321,-0.575057,-0.875000,-0.700000
1965,0.135293,0.126816,0.065875,0.285821,0.201890,0.306685,0.279732,0.342437,0.318323,0.482559,...,1.843744,3.000000,2.950000,2.709086,-0.463836,-0.679577,-0.696809,-0.586957,-0.596899,-0.662791
1966,-0.173722,-0.149883,-0.126769,-0.051098,-0.026934,0.009171,0.183660,0.245741,0.210210,0.280905,...,2.116866,2.062500,1.867722,2.361702,-0.721362,-0.750000,-0.771776,-0.722727,-0.769737,-0.758621
1967,0.154930,0.310548,0.288724,0.657027,0.669991,0.997025,0.307304,0.400564,0.474101,0.658640,...,2.518866,2.903848,2.985507,2.960630,-0.711374,-0.352941,-0.687621,-0.407666,-0.655882,-0.179775
1968,0.143508,0.190218,0.150032,0.272448,0.243661,0.250615,0.298874,0.426980,0.392292,0.541390,...,2.000001,3.000000,2.733333,2.865979,-0.549598,-0.684211,-0.793883,-0.689861,-0.840349,-0.669091


In [303]:
C11=pd.DataFrame(C10.mean())
C11.columns=["Average"]
C11

,Average
B/H mean,-0.015859
S/H mean,-0.063786
B/M mean,0.104643
S/M mean,0.104410
B/L mean,0.242900
S/L mean,0.098363
B/H sd,0.324064
S/H sd,0.457578
B/M sd,0.384213
S/M sd,0.547477


In [45]:
usp=pd.read_csv("/Users/yananmo/Desktop/MGT 930/Excersie 1/US returns.csv")
#drop row with Na values
usp = usp.dropna().reset_index(drop=True)

In [46]:
usp

,gvkey,iid,datadate,tic,conm,prccm,trt1m,exchg
0,1000,01,19701031,AE.2,A & E PLASTIK PAK INC,12.500,6.3830,12.0
1,1000,01,19701130,AE.2,A & E PLASTIK PAK INC,9.000,-28.0000,12.0
2,1000,01,19701231,AE.2,A & E PLASTIK PAK INC,10.000,11.1111,12.0
3,1000,01,19710131,AE.2,A & E PLASTIK PAK INC,9.000,-10.0000,12.0
4,1000,01,19710228,AE.2,A & E PLASTIK PAK INC,11.125,23.6111,12.0
...,...,...,...,...,...,...,...,...
7287453,352257,01C,20220731,HX.,HELIOSX LITHIUM & TECHNOLOGI,0.550,-8.3333,9.0
7287454,352257,01C,20220831,HX.,HELIOSX LITHIUM & TECHNOLOGI,0.760,38.1818,9.0
7287455,352257,01C,20220930,HX.,HELIOSX LITHIUM & TECHNOLOGI,0.640,-15.7895,9.0
7287456,353444,90,20220831,HLN,HALEON PLC,5.980,-14.9360,11.0


In [57]:
usp["datadate"] = pd.to_datetime(usp["datadate"], format="%Y%m%d")

In [330]:
year1=usp["datadate"].dt.year

In [331]:
usp["year"]=year1
usp

,gvkey,iid,datadate,tic,conm,prccm,trt1m,exchg,year
0,1000,01,1970-10-31,AE.2,A & E PLASTIK PAK INC,12.500,6.3830,12.0,1970
1,1000,01,1970-11-30,AE.2,A & E PLASTIK PAK INC,9.000,-28.0000,12.0,1970
2,1000,01,1970-12-31,AE.2,A & E PLASTIK PAK INC,10.000,11.1111,12.0,1970
3,1000,01,1971-01-31,AE.2,A & E PLASTIK PAK INC,9.000,-10.0000,12.0,1971
4,1000,01,1971-02-28,AE.2,A & E PLASTIK PAK INC,11.125,23.6111,12.0,1971
...,...,...,...,...,...,...,...,...,...
7287453,352257,01C,2022-07-31,HX.,HELIOSX LITHIUM & TECHNOLOGI,0.550,-8.3333,9.0,2022
7287454,352257,01C,2022-08-31,HX.,HELIOSX LITHIUM & TECHNOLOGI,0.760,38.1818,9.0,2022
7287455,352257,01C,2022-09-30,HX.,HELIOSX LITHIUM & TECHNOLOGI,0.640,-15.7895,9.0,2022
7287456,353444,90,2022-08-31,HLN,HALEON PLC,5.980,-14.9360,11.0,2022


In [ ]:
C12=pd.DataFrame()
for f in range(1970,2022):
    m2=Cb.loc[f]  
    print('ff')
    for i in range(0,len(usp)):
        if usp.iloc[i,3] in m2['B/H-name'].values and f == usp.iloc[i,8]:
            C12=pd.concat([C12,usp.iloc[i]]) 

ff
ff
ff
ff
ff
ff
ff
ff
ff
ff


In [ ]:
CSH=pd.DataFrame()
for f in range(1970,2022):
    m2=Cb.loc[f]  
    print('ff')
    for i in range(0,len(usp)):
        if usp.iloc[i,3] in m2['S/H-name'].values and f == usp.iloc[i,8]:
            CSH=pd.concat([CSH,usp.iloc[i]]) 

In [ ]:
CBM=pd.DataFrame()
for f in range(1970,2022):
    m2=Cb.loc[f]  
    print('ff')
    for i in range(0,len(usp)):
        if usp.iloc[i,3] in m2['B/M-name'].values and f == usp.iloc[i,8]:
            CBM=pd.concat([CBM,usp.iloc[i]]) 

In [ ]:
CSM=pd.DataFrame()
for f in range(1970,2022):
    m2=Cb.loc[f]  
    print('ff')
    for i in range(0,len(usp)):
        if usp.iloc[i,3] in m2['S/M-name'].values and f == usp.iloc[i,8]:
            CSM=pd.concat([CSM,usp.iloc[i]]) 

In [ ]:
CBL=pd.DataFrame()
for f in range(1970,2022):
    m2=Cb.loc[f]  
    print('ff')
    for i in range(0,len(usp)):
        if usp.iloc[i,3] in m2['B/L-name'].values and f == usp.iloc[i,8]:
            CBL=pd.concat([CBL,usp.iloc[i]]) 

In [ ]:
CSL=pd.DataFrame()
for f in range(1970,2022):
    m2=Cb.loc[f]  
    print('ff')
    for i in range(0,len(usp)):
        if usp.iloc[i,3] in m2['S/L-name'].values and f == usp.iloc[i,8]:
            CSL=pd.concat([CSL,usp.iloc[i]]) 